## Load data

In [9]:
import os
from tqdm import tqdm
import numpy as np
from datasets import load_dataset # huggingface datasets
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

num_proc = 8
subset = '2022_may'
data_folder = f'/home/jan/projects/LMStartup/data/cc_czech/{subset}/'
trainfile = f'{data_folder}train.txt'
testfile = f"{data_folder}test.txt"
# this will be saved in HuggingFace's cache folder
dataset = load_dataset("text", data_files={
    "train": trainfile,
    "test": testfile})
dataset['val'] = dataset.pop('test') # rename the test split to val

Using custom data configuration default-294b79421dff6f07


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 2087.76it/s]


Dataset text downloaded and prepared to /home/jan/.cache/huggingface/datasets/text/default-294b79421dff6f07/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 191.28it/s]


## Train tokenizer

In [10]:
sample_size = 10000
def get_training_corpus():
    for i in range(0, len(dataset['train']),sample_size):
        yield dataset['train'][i:i+sample_size]["text"]

tokenization_type = 'BPE'
special_tokens = ["<unk>", "<|endoftext|>"]

if tokenization_type == 'WORD':
    # word level tokenizer
    tokenizer = Tokenizer(models.WordLevel(unk_token="<unk>"))
    # loqwercase and strip accents
    tokenizer.normalizer = normalizers.Sequence(
        [normalizers.Lowercase(), normalizers.StripAccents()]
    )
    # split on whitespace and punctuation
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
        [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
    )
    trainer = trainers.WordLevelTrainer( vocab_size=500000, special_tokens=special_tokens)

elif tokenization_type == 'BPE':
    tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
    tokenizer.normalizer = normalizers.Sequence(
        [normalizers.Lowercase(), normalizers.StripAccents()]
    )
    tokenizer.decoder = decoders.ByteLevel()
    trainer = trainers.BpeTrainer(vocab_size=50048, special_tokens=special_tokens)


tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)
print("Slovník má velikost: ", len(tokenizer.get_vocab()))




Slovník má velikost:  50048


In [6]:
# ukázka tokenizace
snt = tokenizer.encode('petr pavel bude následující prezident.')
print(snt.tokens)
tokenizer.decode(snt.ids)


['Ġpetr', 'Ġpavel', 'Ġbude', 'ĠnÃ¡sledujÃŃcÃŃ', 'Ġprezident', '<unk>']


' petr pavel bude následující prezident'

## Save tokenizer 

In [11]:
# create a directory if it doesn't exist
save_folder = f'../temp/cc/{subset}'
os.makedirs(save_folder, exist_ok=True)
save_path = f'{save_folder}/tokenizer{tokenization_type.upper()}.json'
tokenizer.save(save_path)

## Tokenize

In [12]:
tokenizer = Tokenizer.from_file(save_path)

def process(example):
    enc = tokenizer.encode(example['text']+"<|endoftext|>") 
    out = {'ids': enc.ids, 'len': len(enc.ids)}
    return out

# tokenize the dataset
tokenized = dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
    num_proc=num_proc,
    load_from_cache_file=False,
)


for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'])
    filename = f'{save_folder}/{split}.bin'
    dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))

    print(f"writing {filename}...")
    idx = 0
    for example in tqdm(dset):
        arr[idx : idx + example['len']] = example['ids']
        idx += example['len']
    arr.flush()

tokenizing the splits #0:   0%|          | 0/143981 [00:00<?, ?ex/s]




















tokenizing the splits #0:   0%|          | 218/143981 [00:00<01:05, 2178.72ex/s]




















tokenizing the splits #0:   0%|          | 459/143981 [00:00<01:02, 2313.61ex/s]




















tokenizing the splits #0:   0%|          | 716/143981 [00:00<00:59, 2427.92ex/s]




















tokenizing the splits #0:   1%|          | 988/143981 [00:00<00:56, 2542.83ex/s]




















tokenizing the splits #0:   1%|          | 1243/143981 [00:00<00:57, 2461.43ex/s]




















tokenizing the splits #0:   1%|          | 1506/143981 [00:00<00:56, 2514.74ex/s]




















tokenizing the splits #0:   1%|          | 1766/143981 [00:00<00:55, 2541.74ex/s]




















tokenizing the splits #0:   1%|▏         | 2021/143981 [00:00<00:56, 2500.83ex/s]




















tokenizing the splits #0:   2%|▏         | 2278/143981 [00:00<00:56, 2515.86ex/s]



















writing ../temp/cc/2022_may/train.bin...


100%|██████████| 1151841/1151841 [00:57<00:00, 19863.71it/s]


writing ../temp/cc/2022_may/val.bin...


100%|██████████| 7387/7387 [00:00<00:00, 20203.50it/s]
